<a href="https://colab.research.google.com/github/Raniamea/arabic-video-summarisation/blob/main/notebooks/02_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install YOLOv8
!pip install -q ultralytics
from ultralytics import YOLO


In [ ]:
# Load YOLOv8 model
model = YOLO("yolov8n.pt")  # Use 'n' for speed, 's' or 'm' for better accuracy


In [ ]:
import os

# Input and output folders
input_dir = "/content/drive/MyDrive/ArabicVideoSummariser/videos"
output_dir = "/content/drive/MyDrive/ArabicVideoSummariser/objects"
os.makedirs(output_dir, exist_ok=True)

# Loop through all video files
video_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.mp4', '.avi', '.mov'))]

for filename in video_files:
    input_path = os.path.join(input_dir, filename)
    print(f"🔍 Processing: {filename}")

    # Run object detection
    results = model.predict(
        source=input_path,
        save=True,
        save_txt=True,
        project=output_dir,
        name=os.path.splitext(filename)[0],  # Create a folder with video name
        exist_ok=True
    )

    print(f"✅ Saved results for: {filename}")
